<a href="https://colab.research.google.com/github/andreiklychev1/Netology-math/blob/master/dogs_vs_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dogs vs Cats

https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition

In [2]:
!pip install pyunpack
!pip install patool

In [3]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import vgg16
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

from pyunpack import Archive

# print(tf.__version__)
# print(tf.executing_eagerly())

Mounted at /content/drive


## Функции загрузки данных

In [4]:
Archive('/content/drive/MyDrive/test.zip').extractall('test')
Archive('/content/drive/MyDrive/train.zip').extractall('train')

In [ ]:
# IMG_SIZE = (224, 224)

# t = train_files[0]
# img = cv2.imread(t)[..., ::-1]

# # datagen = ImageDataGenerator(width_shift_range=[-200,200])


# t = load_img(train_files[0])
# t = img_to_array(t)[..., ::-1]
# img = cv2.resize(t, IMG_SIZE)

# samples = np.expand_dims(t, 0)
# it = datagen.flow(samples, batch_size=1)

In [ ]:
# for i in range(9):
# 	# define subplot
# 	plt.subplot(330 + 1 + i)
# 	# generate batch of images
# 	batch = it.next()
# 	# convert to unsigned integers for viewing
# 	image = batch[0].astype('uint8')
# 	# plot raw pixel data
# 	plt.imshow(image)

In [23]:
from glob import glob
import pandas as pd

train_files = glob('train/train/*.jpg')
test_files = glob('test/test/*.jpg')

train_files = [x.split('/')[2] for x in train_files]
test_files = [x.split('/')[2] for x in test_files]

y = ['1' if os.path.basename(_).startswith('dog') else '0' for _ in train_files]
y1 = ['1' if os.path.basename(_).startswith('dog') else "0" for _ in test_files]

data_train = pd.DataFrame({'path': train_files, 'class': y})
data_test = pd.DataFrame({'path': test_files, 'class': y1})




In [26]:
data_train

,path,class
0,cat.5214.jpg,0
1,dog.3971.jpg,1
2,dog.6780.jpg,1
3,cat.3816.jpg,0
4,dog.581.jpg,1
...,...,...
24995,dog.7572.jpg,1
24996,dog.10562.jpg,1
24997,dog.4911.jpg,1
24998,cat.10.jpg,0


In [24]:
from sklearn.model_selection import train_test_split


train_data, valid_data = train_test_split(data_train, test_size=0.2, random_state = 42)
train_data = train_data.reset_index(drop=True)
valid_data = valid_data.reset_index(drop=True)

In [28]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)




train_generator = datagen.flow_from_dataframe(
    train_data, 
    "train/train", 
    x_col='path',
    y_col='class',
    target_size=(224,224),
    class_mode='categorical',
    batch_size=32
)

Found 20000 validated image filenames belonging to 2 classes.


In [42]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    data_test, 
    "test/test", 
    x_col='path',
    y_col=None,
    class_mode=None,
    target_size=(224,224),
    batch_size=32,
    shuffle=False
)

Found 12500 validated image filenames.


In [29]:
valid_data_gen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_data_gen.flow_from_dataframe(valid_data, 
    "train/train", 
    x_col='path',
    y_col='class',
    target_size=(224,224),
    class_mode='categorical',
    batch_size=32
)

Found 5000 validated image filenames belonging to 2 classes.


In [5]:
# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)

# def aug(el):
#   el = load_image(el)
#   samples = np.expand_dims(el, 0)
#   it = datagen.flow(samples, batch_size=1)
#   batch = it.next()
#   return batch

# def prepare(files):
  
#     for i in range(int(len(files)/32)+1):
#       a = 32
#       try:
#         # x = []
#         # y = []
#         sl = files[a*i:a*(i+1)]
#         x = np.array([load_image(_) for _ in sl])
#         y = np.array([1 if os.path.basename(_).startswith('dog') else 0 for _ in sl])
#         # for s in range(len(x)):
#         #   for t in range(2):
#         #     x1 = np.concatenate((x, aug(x[s])), axis=0)
#         #     y1 = np.concatenate((y, (1 if os.path.basename(x[s]).startswith('dog') else 0)), axis=0)
#         # x = np.asarray(x).astype(np.float32)
#         # y = np.asarray(y).astype(np.float32)
#       except:
#         x = []
#         y = []
#         # sl = train_files[a*i:]
#         # for _ in range(len(sl)):
#           # x.append(load_image(sl[_]))
#           # y.append([1 if os.path.basename(sl[_]).startswith('dog') else 0][0])
#           # for t in range(9):
#             # x.append(aug(sl[_]))
#             # y.append([1 if os.path.basename(sl[_]).startswith('dog') else 0][0])
        
#         x = np.asarray(x).astype(np.float32) 
#         y = np.asarray(y).astype(np.float32) 
#       yield(x, y)

## Загружаем предобученную модель

In [33]:
# base_model - объект класса keras.models.Model (Functional Model)
base_model = vgg16.VGG16(weights='imagenet',
                         include_top=False,
                         input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
# base_model.summary()

## Добавляем полносвязный слой

In [34]:

tf.keras.layers.Flatten()(base_model.layers[-5].output)

<KerasTensor: shape=(None, 100352) dtype=float32 (created by layer 'flatten_1')>

In [35]:
# фиксируем все веса предобученной сети
for layer in base_model.layers:
    layer.trainable = False

x = base_model.layers[-5].output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1,  # один выход (бинарная классификация)
                          activation='sigmoid',  # функция активации  
                          kernel_regularizer=tf.keras.regularizers.l1(1e-4))(x)

model = tf.keras.Model(inputs=base_model.input, outputs=x, name='dogs_vs_cats')

## Выводим архитектуру модели

In [ ]:
# model.summary()

## Компилируем модель и запускаем обучение

In [36]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',  # функция потерь binary_crossentropy (log loss
              metrics=['accuracy'])

In [41]:
# model.fit(next(train_data)[0], next(train_data)[1],
#           steps_per_epoch=10, epochs=100, validation_split=0.1)

model.fit(train_generator, epochs=20, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/20
625/625 [==============================] - 281s 450ms/step - loss: 361.0115 - accuracy: 0.5000 - val_loss: 5.0944 - val_accuracy: 0.5000
Epoch 2/20
625/625 [==============================] - 288s 461ms/step - loss: 386.3846 - accuracy: 0.5000 - val_loss: 5.0964 - val_accuracy: 0.5000
Epoch 3/20
625/625 [==============================] - 284s 455ms/step - loss: 406.7425 - accuracy: 0.5000 - val_loss: 6.3987 - val_accuracy: 0.5000
Epoch 4/20
625/625 [==============================] - 280s 448ms/step - loss: 408.4868 - accuracy: 0.5000 - val_loss: 6.7416 - val_accuracy: 0.5000
Epoch 5/20
625/625 [==============================] - 283s 454ms/step - loss: 415.2978 - accuracy: 0.5000 - val_loss: 7.9928 - val_accuracy: 0.5000
Epoch 6/20
625/625 [==============================] - 280s 449ms/step - loss: 430.8585 - accuracy: 0.5000 - val_loss: 5.8024 - val_accuracy: 0.5000
Epoch 7/20
625/625 [==============================] - 272s 435ms/step - loss: 427.2626 - accuracy: 0.5000 - val_

In [ ]:
model.save('cats-dogs-vgg16 (large).hdf5')

## Предсказания на проверочной выборке

In [ ]:
# test_pred = model.predict(
#     predict_generator(test_files), steps=len(test_files))
test_data = prepare(test_files)
test_pred = model.predict(next(test_data)[0])

In [45]:
predict = model.predict(test_generator, steps=np.ceil(data_test.shape[0]/15))

In [46]:
data_test['class'] = np.argmax(predict, axis=-1)


In [47]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
data_test['class'] = data_test['class'].replace(label_map)

In [48]:
data_test['class'] = data_test['class'].replace({ 'dog': 1, 'cat': 0 })


In [49]:
submission_df = data_test.copy()
submission_df['id'] = submission_df['path'].str.split('.').str[0]
submission_df['label'] = submission_df['class']
submission_df.drop(['path', 'class'], axis=1, inplace=True)
submission_df.to_csv('submission.csv', index=False)